<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#NCSES-class---FedRePORTER-and-IPEDS-data" data-toc-modified-id="NCSES-class---FedRePORTER-and-IPEDS-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>NCSES class - FedRePORTER and IPEDS data</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Python-Setup" data-toc-modified-id="Python-Setup-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Python Setup</a></span></li></ul></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the data</a></span><ul class="toc-item"><li><span><a href="#Federal-RePORTER---Projects-(https://federalreporter.nih.gov/FileDownload)" data-toc-modified-id="Federal-RePORTER---Projects-(https://federalreporter.nih.gov/FileDownload)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Federal RePORTER - Projects (<a href="https://federalreporter.nih.gov/FileDownload" target="_blank">https://federalreporter.nih.gov/FileDownload</a>)</a></span></li><li><span><a href="#Federal-RePORTER---Abstracts-(https://federalreporter.nih.gov/FileDownload)" data-toc-modified-id="Federal-RePORTER---Abstracts-(https://federalreporter.nih.gov/FileDownload)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Federal RePORTER - Abstracts (<a href="https://federalreporter.nih.gov/FileDownload" target="_blank">https://federalreporter.nih.gov/FileDownload</a>)</a></span></li></ul></li><li><span><a href="#Filter-the-data" data-toc-modified-id="Filter-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Filter the data</a></span></li><li><span><a href="#Text-analysis-(Topic-modeling)" data-toc-modified-id="Text-analysis-(Topic-modeling)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Text analysis (Topic modeling)</a></span><ul class="toc-item"><li><span><a href="#NMF-method---Non-negative-matrix-factorization" data-toc-modified-id="NMF-method---Non-negative-matrix-factorization-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>NMF method - Non-negative matrix factorization</a></span></li></ul></li></ul></div>

## NCSES class - FedRePORTER and IPEDS data

### Introduction

**Federal RePORTER** (https://federalreporter.nih.gov) - a collaborative effort led by STAR METRICS® to create a searchable database of scientific awards from agencies (across agencies or fiscal years, by the award's project leader, or by a text search of a project's title, terms, or abstracts).

### Python Setup

In [1]:
# Data manipulation
import pandas as pd

# Reading in files
import glob

# Text analysis (topic modeling)
import numpy as np
import sklearn
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

## Load the data

### Federal RePORTER - Abstracts (https://federalreporter.nih.gov/FileDownload)

In [2]:
abstracts_files = glob.glob('FedRePORTER_PRJABS_C_FY20*.csv')
print(abstracts_files)

['FedRePORTER_PRJABS_C_FY2009.csv', 'FedRePORTER_PRJABS_C_FY2008.csv', 'FedRePORTER_PRJABS_C_FY2018.csv', 'FedRePORTER_PRJABS_C_FY2017.csv', 'FedRePORTER_PRJABS_C_FY2003.csv', 'FedRePORTER_PRJABS_C_FY2002.csv', 'FedRePORTER_PRJABS_C_FY2016.csv', 'FedRePORTER_PRJABS_C_FY2000.csv', 'FedRePORTER_PRJABS_C_FY2014.csv', 'FedRePORTER_PRJABS_C_FY2015.csv', 'FedRePORTER_PRJABS_C_FY2001.csv', 'FedRePORTER_PRJABS_C_FY2005.csv', 'FedRePORTER_PRJABS_C_FY2011.csv', 'FedRePORTER_PRJABS_C_FY2010.csv', 'FedRePORTER_PRJABS_C_FY2004.csv', 'FedRePORTER_PRJABS_C_FY2012.csv', 'FedRePORTER_PRJABS_C_FY2006.csv', 'FedRePORTER_PRJABS_C_FY2007.csv', 'FedRePORTER_PRJABS_C_FY2013.csv']


In [3]:
"""Read them in, concatenate and convert to a dataframe."""

list_data = []
for filename in abstracts_files:
    data = pd.read_csv(filename)
    list_data.append(data)
    
abstracts = pd.concat(list_data)

In [4]:
abstracts = abstracts.dropna()

In [5]:
merged_abstracts_list = abstracts[' ABSTRACT'].values.tolist()

### NMF method - Non-negative matrix factorization

NMF is another model used for topic extraction - while the LDA model uses raw counts of unique words per document, NMF model uses a normalized representation of those raw counts (TF-IDF representation)

TF stands for term-frequency and TF-IDF is term-frequency times inverse document-frequency. In other words, we are not only looking for how often a word appears in a given document, but also whether this particular word is distinct across all the collections of documents (corpus). For example, intuitively we understand that words like "often" or "use" are more frequently encountered, but they are less informative (more semantically-vacuous) if we want to discern a particular topic of a document, as they might be frequently encounter across all text documents in a corpus. On the other hand, words which we will see less frequently across a collection of document might indicate that those words are specific to a particular document, and, therefore, constitute a basis for a topic. 

More here: 

- https://scikit-learn.org/stable/modules/decomposition.html#non-negative-matrix-factorization-nmf-or-nnmf
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer

In [12]:
# Convert a collection of raw documents to a matrix of TF-IDF features
vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(filtered_abstracts)

In [13]:
# Get feature names
vectorizer_feature_names = vectorizer.get_feature_names()

In [ ]:
# Run the model with 10 topics
nmf = NMF(n_components=100, verbose=2).fit(tfidf)

In [15]:
nmf_100_2 = nmf

In [16]:
%store nmf_100_2

Stored 'nmf_100_2' (NMF)


In [6]:
%store -r nmf_100

In [9]:
nmf_W = nmf_100_2.transform(tfidf) 

In [10]:
topics_weights = []
for index,i in enumerate(nmf_W): # for every document
    topics_weights.append([index, i[25], i[57]]) # get topic weights

In [11]:
topics_list_dataframe = pd.DataFrame(topics_weights)
topics_list_dataframe.head()

,0,1,2
0,0,0.000000,0.0
1,1,0.000000,0.0
2,2,0.000000,0.0
3,3,0.000108,0.0
4,4,0.000000,0.0


In [12]:
abstracts = abstracts.reset_index()

In [ ]:
topics_list_dataframe = topics_list_dataframe.rename(columns={0:'index'})

In [24]:
topics_list_dataframe.index.nunique()

1032895

In [25]:
abstracts.index.nunique()

1032895

In [13]:
concat = pd.concat([abstracts,topics_list_dataframe],axis=1)

In [14]:
concat.head()

,index,PROJECT_ID,ABSTRACT,0,1,2
0,0,103915,EDUCATION IN ACTION NASA Exchange City Learnin...,0,0.000000,0.0
1,1,103916,Educational Advancement Alliance Inc Math Scie...,1,0.000000,0.0
2,2,103917,"CUBRC, Inc FY09 Earmark Entitled, to continue...",2,0.000000,0.0
3,3,103918,University Corporation for Atmospheric Researc...,3,0.000108,0.0
4,4,103919,Proposal Number: 0850898PI: John Doyle ...,4,0.000000,0.0


In [15]:
filtered = concat[(concat[1] != 0) | (concat[2] != 0)]

In [16]:
len(filtered)

416312

In [26]:
filtered.to_csv('filtered_results.csv')

In [8]:
filtered = pd.read_csv('filtered_results.csv')

In [9]:
filtered_abstracts = filtered[' ABSTRACT'].values.tolist()

In [10]:
len(filtered_abstracts)

416312

In [30]:
pd.set_option('display.max_colwidth', -1)

In [19]:
opioid_terms = ['opioid','opiate','morphine','heroin',
                'percocet','vicoprofen','dextromethorphan','loperamide',
                'naloxegol','hydrocodone','oxycodone','fentanyl',
                'naloxone','analgesics','carfentanil','benzodiazepines',
                'narcotic','opium','cocaine','codeine',
                'pain relief','cancer pain','anesthesia','chronic pain',
                'nerve pain','fibromyalgia','overdose','addiction',
                'withdrawal','dependence','recreational use','euphoria',
                'tolerance','controlled substance','over-prescription',
                'peripheral nervous system','psychoactive','agonist',
                'antagonist','blood-brain']

In [21]:
results = []
for term in opioid_terms:
    for abstract in filtered[' ABSTRACT']:
        results.append([term,abstract])

In [ ]:
len(results)

In [24]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
results = pd.DataFrame(results)
results

In [23]:
filtered = topics_list_dataframe[(topics_list_dataframe[1] != 0) | (topics_list_dataframe[2] != 0)]

In [31]:
filtered = filtered.rename(columns={0:'index'})

In [37]:
filtered = filtered.reset_index()

In [ ]:
abstracts = abstracts.reset_index()

In [39]:
topics_abstracts = abstracts.merge(filtered,on='index')

In [17]:
nmf_W = nmf_100_2.transform(tfidf) # get topics to documents matrix
nmf_H = nmf_100_2.components_ # get word to topics matrix

violation: 1.0
violation: 0.5176337571181686
violation: 0.050164562381222834
violation: 0.011162182993669261
violation: 0.0025927295912233244
violation: 0.0006365453148000184
violation: 0.00018226397383737058
violation: 4.611443097461428e-05
Converged at iteration 8


In [20]:
# View the list of topics (10 top words per topic)

for topic_idx, topic in enumerate(nmf_H):
    print("Topic %d:" % (topic_idx))
    print('----------------------------')
    print(" ".join([vectorizer_feature_names[i]
                for i in topic.argsort()[:-10 - 1:-1]]))
    print('----------------------------')

Topic 0:
----------------------------
genetic genes variants genome variation identify genetics traits association genomic
----------------------------
Topic 1:
----------------------------
research investigators translational support development university biomedical researchers new career
----------------------------
Topic 2:
----------------------------
intervention interventions adherence randomized trial group based efficacy months participants
----------------------------
Topic 3:
----------------------------
subproject institution nih center isfor andinvestigator crisp theresources entries necessarily
----------------------------
Topic 4:
----------------------------
hiv infected aids infection antiretroviral art msm prevention transmission risk
----------------------------
Topic 5:
----------------------------
cancer nci cancers pancreatic colon prevention colorectal survivors members oncology
----------------------------
Topic 6:
----------------------------
core projects inve

decision making decisions choice information reward choices make policy value
----------------------------
Topic 58:
----------------------------
animal animals models human primate mice laboratory model mouse studies
----------------------------
Topic 59:
----------------------------
stem differentiation renewal hematopoietic hsc cells niche hscs progenitor adult
----------------------------
Topic 60:
----------------------------
prostate men pca cancer androgen ar prostatic progression african bph
----------------------------
Topic 61:
----------------------------
clinical trials trial studies translational phase protocol medical conduct safety
----------------------------
Topic 62:
----------------------------
kidney renal ckd hypertension disease blood pressure chronic cardiovascular vascular
----------------------------
Topic 63:
----------------------------
program programs members year programmatic funding scientific biomedical departments progress
----------------------------
T

In [22]:
# View a top document related to a given topic

for topic_idx, topic in enumerate(nmf_H):
    print('--------------------')
    print("Topic %d:" % (topic_idx))
    print('--------------------')
    print(" ".join([vectorizer_feature_names[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
    top_doc_indices = np.argsort(nmf_W[:,topic_idx] )[::-1][0:1]
    for doc_index in top_doc_indices:
        print('--------------------')
        print(filtered_abstracts[doc_index])

--------------------
Topic 0:
--------------------
genetic genes variants genome variation identify genetics traits association genomic
--------------------
The goal of the laboratory is to develop new approaches to the study of the genetic basis of cancer and its outcomes. Previously, the major focus was the analysis of common genetic variation in candidate genes in cancer and its related outcomes, particularly in immunocompromised individuals. Emphasis was on conducting pilot association studies and annotating candidate genes drawn from key pathways in innate immunity and cancer biology, such as telomere stability or nutrient transport (i.e., Vitamin C sodium dependent transport). The laboratory has developed expertise in bio-informatics and advanced genetic analyses with new platforms designed to test dense sets of single nucleotide polymorphisms (SNPs), which are the most common genetic variants in the human genome. Specifically, the laboratory has integrated approaches to identify

hiv infected aids infection antiretroviral art msm prevention transmission risk
--------------------
DESCRIPTION (provided by applicant): HIV dementia (HIV-D) and HIV-associated sensory neuropathy (HIV-SN) are the most common neurological manifestations of advanced HIV infection. The prevalence of HIV-D and HIV-SN in Sub- Saharan Africa where the majority of HIV cases reside globally is largely unknown. In addition, HIV subtype may have an impact on HIV disease progression, suggesting the possibility that HIV subtypes may differ with respect to their ability to cause neurological disease. The project will assemble a cohort of HIV+ individuals in Uganda: 1) to determine the prevalence of and risk factors associated with HIV-D and HIV-SN among untreated HIV+ individuals with moderate advanced immunosuppression, 2) to determine whether untreated HIV+ individuals decline from baseline in neuropsychological test performance, and peripheral nerve function, and 3) to obtain preliminary data t

--------------------
DESCRIPTION (provided by applicant): Older adults are at increased risk to develop prolonged pain and experience greater pain-related loss of physical and psychosocial function compared to younger cohorts. We propose that changes in endogenous pain modulatory capacity accounts for increased incidence of pain and disability in older adults. That endogenous pain modulation dysfunction is related to persistent pain is supported by a number of studies comparing chronic pain patients with healthy controls using a  pain-inhibition-by-pain  experimental model. Although this research group (and others) has shown age deficiencies using the  pain inhibits pain  model, other human laboratory models that are known to engage pain modulatory systems have not been tested across the lifespan, each potentially involving different mechanisms. The overarching goal of the proposed research is to characterize age-related changes in pain inhibitory and facilitatory function and to inves

--------------------
DESCRIPTION (provided by applicant): My goal for the K25 award is to establish myself as an independent neuroimaging researcher with expertise in brain network analysis and an integral member of multidisciplinary research teams devoted to addressing diseases of the brain. Attaining these objectives will require focused didactic training and research guidance. Research We will develop new methodology to improve whole-brain connectivity analyses of normal and abnormal brain function. The launching of the Human Connectome Project by the NIH in 2009 underscores the importance of whole-brain connectivity analyses. Appropriately conducting these analyses is paramount in our understanding normal brain function as well as alterations due to conditions such as aging, dyslexia, and substance abuse. Before we can glean useful information from functional brain network differences in these conditions, methods need to be developed in order to permit 1) assessing several network 

--------------------
DESCRIPTION (provided by applicant): Evidence for the effectiveness of community-based psychosocial treatments for adolescent substance use disorders is mixed, at best. Researchers' inability to detect strong or replicable treatment effects may result from their focus on the post-treatment effects of a single incident treatment episode. Recent conceptualizations of addiction and treatment suggest that these approaches which compare pre-post treatment effects may obscure some of treatment's most salient effects. For one, treatment effects should be expected to be greatest during treatment or concurrently. Second, from a treatment careers perspective, multiple treatment episodes over time are likely to lead to cumulative effects, which would be greater than those observed for any individual treatment episode. Finally, considerations of client heterogeneity suggest that effects of treatment may be greatest for a subgroup of patients (moderated effects), with such effe

...

ptsd veterans tbi va fear trauma traumatic symptoms extinction military
--------------------
DESCRIPTION (provided by applicant):        Posttraumatic Stress Disorder (PTSD), a highly prevalent, chronic psychiatric disorder, often co-occurs with other psychiatric disorders, is associated with occupational and social dysfunction, and can lead to chronic disability. PTSD is also a risk factor for a number of health-related concerns including chronic pain and cardiovascular disease and is associated with significant economic burden. Veterans are at increased risk for PTSD with prevalence rates up to 3 times those who are non-Veterans. Existing PTSD treatments have been found to be less effective for Veterans with PTSD, in fact, combat-related trauma has been associated with the lowest treatment effect sizes in studies of PTSD treatments. Veterans with PTSD often experience difficulty engaging in traditional CBT exposure based approaches, which can trigger the uncomfortable emotional arous

--------------------
DESCRIPTION (provided by applicant): The overall goal of The Clinical Profile of Parkinson's Disease (PD) Pathology, is to characterize the clinical profile of PD pathology in older person's without a diagnosis of PD. Showing that PD pathology is associated with a distinct and progressive condition among persons without a clinical diagnosis of PD, would have a transformative effect on PD studies. Although, PD only affects up to 5% of persons by age 85, compelling preliminary data shows that indices of PD pathology including nigral degeneration and Lewy bodies are present in nearly 20% of older persons without PD and are associated with the severity of parkinsonism proximate to death. This suggests that PD pathology is common and causes clinical signs in persons who do not meet clinical criteria for PD. Like the recent reclassification of AD, PD may also have an asymptomatic PD pathology phase, followed by a stage in which PD pathology results in mild motor and non-

mitochondrial mitochondria ros dysfunction oxidative oxygen mtdna energy iron metabolism
--------------------
﻿   DESCRIPTION (provided by applicant): The impact of mitochondrial biology on human cancers is broad because these organelles are critical regulators of metabolism, proliferation, and apoptosis. Indeed, mitochondrial aberrations are common in multiple cancer types --- not only do mitochondrial dysfunctions correlate with disease pathogenesis, but aberrant mitochondria also negatively impact upon chemotherapeutic success. Within a cell, mitochondrial homeostasis is maintained by a process referred to as  mitochondrial dynamics , which is essential for mitochondrial genome integrity, efficient ATP generation, managing ROS, and the rapid distribution of mitochondrial metabolites. Mitochondrial dynamics result from the cumulative nature of two opposing forces: mitochondrial division and mitochondrial fusion. Recent published work from my group demonstrated: (1) mitochondrial divi

exercise physical aerobic activity training fitness intensity sedentary walking effects
--------------------
﻿   DESCRIPTION (provided by applicant): Given the rising proportion of older adults worldwide and the progressive decline in brain function with advancing age, there is a pressing need to develop novel interventions that protect the aging brain. The predominant approach for implementing exercise training to improve brain function is to increase cardiovascular fitness. However, there is mixed empirical support for the effectiveness of this approach. Further, there are also acute effects of exercise within one hour of the cessation of a single exercise session. These effects occur before adaptations related to fitness could occur and animal studies have shown they occur in the same brain regions that benefit from longer-term exercise training. Therefore, we propose the acute paradigm is a tool to probe this early, direct response from exercise in order to determine how best to ma

oral dental caries hpv tooth periodontal craniofacial mutans taste cavity
--------------------
DESCRIPTION (provided by applicant): Oral health is vital to a person's overall health, and yet the incidence and prevalence of dental problems and oral diseases remain quite high. Statistics show that severe gum disease affects almost 15% of adults, and a large proportion of adults show at least some signs of gum disease. Furthermore, preliminary evidence also indicates that oral health is closely related to systemic diseases through a number of routes such as blood circulation; due the complexity involved with the process and procedures of clinical encounters and the increasing overall costs of health care, dental exam and treatment are more complicated and expensive. The reality is that underserved populations do not have the ability to identify a usual source of care, lack dental insurance to support the provision of care, and are burdened by competing needs. Self-report is the most effic

In [401]:
# Get the index of documents and topic probabilities per document
topics_probabilities = []
for index,i in enumerate(nmf_W): # for every document
    topics_probabilities.append([i, index]) # get all topic probabilities

In [390]:
# Get the index of documents and associated (most probable) topic index
topics_list = []
for index,i in enumerate(nmf_W): # for every document
    topics_list.append([np.argsort(nmf_W[index])[::-1][0], index]) # get most probable topic

In [391]:
topics_list_dataframe = pandas.DataFrame(topics_list)
topics_list_dataframe = topics_list_dataframe.rename(columns={0:'topic',1:'index'})
merged_abstracts = merged_abstracts.reset_index()
merged_topics_abstracts = merged_abstracts.merge(topics_list_dataframe,on='index')

In [ ]:
# View a distribution of abstracts per topic
merged_topics_abstracts.groupby('topic')[' ABSTRACT'].count()

In [ ]:
topics_weights = []
for index,i in enumerate(nmf_W): # for every document
    topics_weights.append([index, i[11], i[25], i[32], i[43]]) # get topic weights for 4 related topics

topics_weights_dataframe = pd.DataFrame(topics_weights)
topics_weights_dataframe = topics_weights_dataframe.set_index(0)
topics_weights_dataframe = topics_weights_dataframe.loc[(topics_weights_dataframe!=0).any(1)]

topics_weights_dataframe.columns = [11,25,32,43]

# Find the maximum value for a given topic
topics_weights_dataframe[topics_weights_dataframe[43]==topics_weights_dataframe[43].max()]

filtered_abstracts = set(filtered_abstracts)

# View a given abstract based on a treshold
list(filtered_abstracts)[556]